In [2]:
import pandas as pd
import numpy as np
from keras.layers import Input,LSTM,Dense,CuDNNLSTM,Bidirectional
from keras.models import Model
from keras.optimizers import Adam,rmsprop

In [3]:
data_path = '../data/cmn.txt'
df = pd.read_table(data_path,header=None)
df.columns=['inputs','targets']

#讲每句中文句首加上'\t'作为起始标志，句末加上'\n'作为终止标志
df['targets'] = df['targets'].apply(lambda x: '\t'+x+'\n')
df.head(5)

,inputs,targets
0,Hi.,\t嗨。\n
1,Hi.,\t你好。\n
2,Run.,\t你用跑的。\n
3,Wait!,\t等等！\n
4,Hello!,\t你好。\n


In [4]:
input_texts = df.inputs.values.tolist()#英文句子列表
target_texts = df.targets.values.tolist()#中文句子列表
print('input texts samples" {}'.format(len(input_texts)))

#确定中英文各自包含的字符。df.unique()直接取sum可将unique数组中的各个句子拼接成一个长句子
input_characters = sorted(list(set(df.inputs.unique().sum())))
target_characters = sorted(list(set(df.targets.unique().sum())))

print('last 5 character of input text: {}'.format(input_characters[-5:]))
print('last 5 character of target text: {}'.format(target_characters[-5:]))



input texts samples" 20294
last 5 character of input text: ['x', 'y', 'z', 'é', '’']
last 5 character of target text: ['，', '－', '：', '？', '𡡡']


## 每条句子经过对字母转换成one-hot编码后，生成了LSTM需要的三维输入[n_samples, timestamp, one-hot feature]

In [5]:
"""
NUM_SAMPLES，样本条数，这里是输入的句子条数
INPUT_LENGTH，输入数据的时刻t的长度，这里为最长的英文句子长度
OUTPUT_LENGTH，输出数据的时刻t的长度，这里为最长的中文句子长度
INPUT_FEATURE_LENGTH，每个时刻进入encoder的lstm单元的数据x t  xtx_t的维度，这里为英文中出现的字符数
OUTPUT_FEATURE_LENGTH，每个时刻进入decoder的lstm单元的数据x t  xtx_t的维度，这里为中文中出现的字符数
"""
NUM_SAMPLES = len(target_texts)
INPUT_LENGTH = max(df.inputs.apply(lambda x: len(x)))
OUTPUT_LENGTH = max(df.targets.apply(lambda x: len(x)))
INPUT_FEATURE_LENGTH = len(input_characters)
OUTPUT_FEATURE_LENGTH = len(target_characters)
print('NUM_SAMPLES: {}'.format(NUM_SAMPLES))
print('INPUT_LENGTH: {}'.format(INPUT_LENGTH))
print('OUTPUT_LENGTH: {}'.format(OUTPUT_LENGTH))
print('INPUT_FEATURE_LENGTH: {}'.format(INPUT_FEATURE_LENGTH))
print('OUTPUT_FEATURE_LENGTH: {}'.format(OUTPUT_FEATURE_LENGTH))

#encoder输入、decoder输入输出初始化为三维向量
encoder_input = np.zeros((NUM_SAMPLES,INPUT_LENGTH,INPUT_FEATURE_LENGTH))
decoder_input = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))
decoder_output = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))


input_dict = {char:index for index,char in enumerate(input_characters)}
input_dict_reverse = {index:char for index,char in enumerate(input_characters)}
target_dict = {char:index for index,char in enumerate(target_characters)}
target_dict_reverse = {index:char for index,char in enumerate(target_characters)}




NUM_SAMPLES: 20294
INPUT_LENGTH: 163
OUTPUT_LENGTH: 46
INPUT_FEATURE_LENGTH: 75
OUTPUT_FEATURE_LENGTH: 3427


# 对句子进行字符级one-hot编码，将输入输出数据向量化：

# question 1: paddle at the end of a scentence ? vs paddle at the beginning of a scentence

In [6]:
#encoder的输入向量one-hot
for seq_index,seq in enumerate(input_texts):
    for char_index, char in enumerate(seq):
        encoder_input[seq_index,char_index,input_dict[char]] = 1

# question 2: decoder_input char_index starts from 0, should not be 1?

In [7]:
#decoder的输入输出向量one-hot，训练模型时decoder的输入要比输出晚一个时间步，这样才能对输出监督
for seq_index,seq in enumerate(target_texts):
    for char_index,char in enumerate(seq):
        decoder_input[seq_index,char_index,target_dict[char]] = 1.0
        if char_index > 0:
            decoder_output[seq_index,char_index-1,target_dict[char]] = 1.0

# MODELING

In [8]:
def create_model(n_input,n_output,n_units):
    """
    n_input: encoder输入维度n_input为每个时间步的输入xt的维度，这里是用来one-hot的英文字符数
    n_output: decoder的输入维度为中文字符数


    """
    #训练阶段
    encoder_input = Input(shape = (None, n_input))
    encoder = CuDNNLSTM(n_units, return_state=True)
    _,encoder_h,encoder_c = encoder(encoder_input)
    encoder_state = [encoder_h,encoder_c]

    #decoder
    decoder_input = Input(shape = (None, n_output))
    decoder = CuDNNLSTM(n_units,return_sequences=True, return_state=True)
    decoder_output, _, _ = decoder(decoder_input,initial_state=encoder_state)
    decoder_dense = Dense(n_output,activation='softmax')
    decoder_output = decoder_dense(decoder_output)

    #生成的训练模型
    model = Model([encoder_input,decoder_input],decoder_output)

    #推理阶段，用于预测过程
    encoder_infer = Model(encoder_input,encoder_state)

    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))    
    decoder_state_input = [decoder_state_input_h, decoder_state_input_c]#上个时刻的状态h,c   

    decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(decoder_input,initial_state=decoder_state_input)
    decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]#当前时刻得到的状态
    decoder_infer_output = decoder_dense(decoder_infer_output)#当前时刻的输出
    decoder_infer = Model([decoder_input]+decoder_state_input,[decoder_infer_output]+decoder_infer_state)

    return model, encoder_infer, decoder_infer


In [65]:
def predict_chinese(source,encoder_inference, decoder_inference, n_steps, features):
    #先通过推理encoder获得预测输入序列的隐状态
    state = encoder_inference.predict(source)
    #第一个字符'\t',为起始标志
    predict_seq = np.zeros((1,1,features))
    predict_seq[0,0,target_dict['\t']] = 1

    output = ''
    #开始对encoder获得的隐状态进行推理
    #每次循环用上次预测的字符作为输入来预测下一次的字符，直到预测出了终止符
    for i in range(n_steps):#n_steps为句子最大长度
        #给decoder输入上一个时刻的h,c隐状态，以及上一次的预测字符predict_seq
        yhat,h,c = decoder_inference.predict([predict_seq]+state)
        #注意，这里的yhat为Dense之后输出的结果，因此与h不同
        char_index = np.argmax(yhat[0,-1,:])
        char = target_dict_reverse[char_index]
        output += char
        state = [h,c]#本次状态做为下一次的初始状态继续传递
        predict_seq = np.zeros((1,1,features))
        predict_seq[0,0,char_index] = 1
        if char == '\n':#预测到了终止符则停下来
            break
    return output


In [66]:
model, encoder_infer, decoder_infer = create_model(n_input=INPUT_FEATURE_LENGTH,
                                                   n_output=OUTPUT_FEATURE_LENGTH,
                                                   n_units=1024)

In [67]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, None, 75)     0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, None, 3427)   0                                            
__________________________________________________________________________________________________
cu_dnnlstm_17 (CuDNNLSTM)       [(None, 1024), (None 4509696     input_33[0][0]                   
__________________________________________________________________________________________________
cu_dnnlstm_18 (CuDNNLSTM)       [(None, None, 1024), 18239488    input_34[0][0]                   
                                                                 cu_dnnlstm_17[0][1]              
          

In [69]:
model.compile(
        loss='categorical_crossentropy',
        optimizer=rmsprop(),
        metrics=['accuracy'])


In [30]:
model.fit(x=[encoder_input,decoder_input],
          y=decoder_output,
          batch_size=64,
          epochs=100,
          validation_split=0.2)

Train on 16235 samples, validate on 4059 samples
Epoch 1/100
16235/16235 [==============================] - 50s 3ms/step - loss: 1.1334 - acc: 0.0375 - val_loss: 1.8815 - val_acc: 0.0503
Epoch 2/100
16235/16235 [==============================] - 49s 3ms/step - loss: 1.0330 - acc: 0.0535 - val_loss: 1.7581 - val_acc: 0.0573
Epoch 3/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.9556 - acc: 0.0622 - val_loss: 1.6578 - val_acc: 0.0657
Epoch 4/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.8960 - acc: 0.0683 - val_loss: 1.6080 - val_acc: 0.0726
Epoch 5/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.8499 - acc: 0.0729 - val_loss: 1.5435 - val_acc: 0.0781
Epoch 6/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.8134 - acc: 0.0770 - val_loss: 1.5219 - val_acc: 0.0819
Epoch 7/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.7831 - acc: 0.0803 - val_loss: 1.5254 - v

16235/16235 [==============================] - 51s 3ms/step - loss: 0.3451 - acc: 0.1470 - val_loss: 1.8585 - val_acc: 0.0867
Epoch 60/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.3422 - acc: 0.1476 - val_loss: 1.8575 - val_acc: 0.0863
Epoch 61/100
16235/16235 [==============================] - 51s 3ms/step - loss: 0.3398 - acc: 0.1480 - val_loss: 1.8607 - val_acc: 0.0869
Epoch 62/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.3373 - acc: 0.1484 - val_loss: 1.8684 - val_acc: 0.0868
Epoch 63/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.3348 - acc: 0.1488 - val_loss: 1.8848 - val_acc: 0.0865
Epoch 64/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.3323 - acc: 0.1492 - val_loss: 1.8839 - val_acc: 0.0868
Epoch 65/100
16235/16235 [==============================] - 50s 3ms/step - loss: 0.3299 - acc: 0.1496 - val_loss: 1.8944 - val_acc: 0.0865
Epoch 66/100
16235/16235 [==============

In [70]:

for j in range(300):
    print(j)
    model.fit(x=[encoder_input,decoder_input],y=decoder_output,batch_size=None)
    i= np.random.randint(10000)
    print(input_texts[i])
    test = encoder_input[i:i+1,:,:]
    out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    print(out)
    print('=============================================')

0
Epoch 1/1
20294/20294 [==============================] - 85s 4ms/step - loss: 1.1801 - acc: 0.0535
I saw him playing baseball.
我們為你的时候，我不知道。

1
Epoch 1/1
20294/20294 [==============================] - 85s 4ms/step - loss: 0.9735 - acc: 0.0765
Why did you open the box?
我們的父親很有趣。

2
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.8870 - acc: 0.0859
I come from England.
我不知道他是什么。

3
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.8302 - acc: 0.0919
I've seen that.
我不想要你。

4
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.7836 - acc: 0.0973
Where is the nearest station?
我們在這裡很多次了。

5
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.7385 - acc: 0.1032
My hands and legs are swollen.
我們的父母是一個很大的問題。

6
Epoch 1/1
20294/20294 [==============================] - 85s 4ms/step - loss: 0.6959 - acc: 0.1089
Tom is a normal teenager.
我不知道他的名字。

7
Epoch 1/1
20294/20294 [======

20294/20294 [==============================] - 84s 4ms/step - loss: 0.3464 - acc: 0.1667
The boy is eating bread.
我們在那裡抓了一些大魚。

45
Epoch 1/1
20294/20294 [==============================] - 85s 4ms/step - loss: 0.3449 - acc: 0.1668
You look very tired.
我不知道他是否愛我。

46
Epoch 1/1
20294/20294 [==============================] - 85s 4ms/step - loss: 0.3440 - acc: 0.1668
Tom isn't dumb.
我們在那裡上了公車汽車。

47
Epoch 1/1
20294/20294 [==============================] - 85s 4ms/step - loss: 0.3422 - acc: 0.1671
Dead men tell no tales.
我們不能在這裡待很長時間。

48
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.3432 - acc: 0.1668
I've got a lot more to learn.
我不想再听到其他借口了。

49
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.3440 - acc: 0.1664
How would you like your steak?
我們在那裡有一個不愉快的。

50
Epoch 1/1
20294/20294 [==============================] - 85s 4ms/step - loss: 0.3432 - acc: 0.1664
Which book is better?
我不想吃早餐。

51
Epoch 1/1
20294/20294 [===========

20294/20294 [==============================] - 83s 4ms/step - loss: 0.3624 - acc: 0.1600
Take it.
我不知道我的手錶在哪裡。

89
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.3607 - acc: 0.1604
The price is not reasonable.
我不知道他是什么时候从法国回来的。

90
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.3617 - acc: 0.1602
Tom is a clever kid.
我們在那裡上了公共汽車。

91
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.3639 - acc: 0.1597
He is a friendly person.
我們在那裡上了太茶。

92
Epoch 1/1
20294/20294 [==============================] - 86s 4ms/step - loss: 0.3628 - acc: 0.1598
I'm innocent.
我不知道他是否會來。

93
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.3632 - acc: 0.1598
She sat on the bench.
我不知道我們為怎麼做。

94
Epoch 1/1
20294/20294 [==============================] - 84s 4ms/step - loss: 0.3635 - acc: 0.1596
Don't forget about me.
我不知道他是否會來。

95
Epoch 1/1
20294/20294 [==============================] - 

KeyboardInterrupt: 

In [44]:
i = 2
test = encoder_input[i:i+1,:,:]
out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
print(input_texts[i])
print(out)

Come in.
我們在這裡花了太多時間。



In [ ]:
def decode_sequence(input_seq):
    # 将输入序列进行编码
    states_value = encoder_model.predict(input_seq)

    # 生成一个size=1的空序列
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 将这个空序列的内容设置为开始字符
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 进行字符恢复
    # 简单起见，假设batch_size = 1
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 退出条件：生成 \n 或者 超过最大序列长度
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length :
            stop_condition = True

        # 更新target_seq
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 更新中间状态
        states_value = [h, c]

    return decoded_sentence
--------------------- 
作者：芥末的无奈 
来源：CSDN 
原文：https://blog.csdn.net/weiwei9363/article/details/79464789 
版权声明：本文为博主原创文章，转载请附上博文链接！

# out

In [67]:
train_lstm = model.get_layer('cu_dnnlstm_5')

In [72]:
print(train_lstm.weights[0])
train_lstm.get_weights()[0]

<tf.Variable 'cu_dnnlstm_5/kernel:0' shape=(75, 256) dtype=float32_ref>


array([[ 0.00048403, -0.10057795,  0.09402207, ...,  0.06421671,
         0.04817443, -0.04124532],
       [ 0.02149276,  0.00541093,  0.11336602, ...,  0.12082654,
        -0.11954425, -0.08038449],
       [ 0.09851375, -0.12272102, -0.11307565, ..., -0.08199231,
        -0.02517138, -0.02650847],
       ...,
       [ 0.06733374,  0.02784984,  0.02314961, ...,  0.09500158,
         0.12246257,  0.00699327],
       [-0.05950085, -0.10947321,  0.0575029 , ..., -0.01508953,
        -0.04106094, -0.06283302],
       [-0.11229177,  0.06529395, -0.0673771 , ..., -0.0839394 ,
        -0.08219849,  0.09753321]], dtype=float32)

In [77]:
infer_lstm = encoder_infer.get_layer('cu_dnnlstm_5')
print(infer_lstm.weights[0])
infer_lstm.get_weights()[0]

<tf.Variable 'cu_dnnlstm_5/kernel:0' shape=(75, 256) dtype=float32_ref>


array([[ 0.00048403, -0.10057795,  0.09402207, ...,  0.06421671,
         0.04817443, -0.04124532],
       [ 0.02149276,  0.00541093,  0.11336602, ...,  0.12082654,
        -0.11954425, -0.08038449],
       [ 0.09851375, -0.12272102, -0.11307565, ..., -0.08199231,
        -0.02517138, -0.02650847],
       ...,
       [ 0.06733374,  0.02784984,  0.02314961, ...,  0.09500158,
         0.12246257,  0.00699327],
       [-0.05950085, -0.10947321,  0.0575029 , ..., -0.01508953,
        -0.04106094, -0.06283302],
       [-0.11229177,  0.06529395, -0.0673771 , ..., -0.0839394 ,
        -0.08219849,  0.09753321]], dtype=float32)

In [79]:
infer_lstm = encoder_infer.get_layer('cu_dnnlstm_5')
print(infer_lstm.weights[0])
infer_lstm.get_weights()[0]

<tf.Variable 'cu_dnnlstm_5/kernel:0' shape=(75, 256) dtype=float32_ref>


array([[ 0.00048537, -0.10057795,  0.09402207, ...,  0.06423962,
         0.04818527, -0.04124521],
       [ 0.02149243,  0.00541081,  0.11336623, ...,  0.12081587,
        -0.11976563, -0.0803909 ],
       [ 0.09846777, -0.12273928, -0.11309011, ..., -0.08199231,
        -0.02517122, -0.02650827],
       ...,
       [ 0.06733384,  0.02784988,  0.02314961, ...,  0.09500158,
         0.12246257,  0.00699364],
       [-0.05950077, -0.10947321,  0.05750281, ..., -0.01508927,
        -0.04100636, -0.06283302],
       [-0.11229177,  0.06529395, -0.0673771 , ..., -0.0839394 ,
        -0.08219849,  0.09753321]], dtype=float32)

In [23]:
model.fit(x=[encoder_input,decoder_output],y=decoder_input,batch_size=512)

Epoch 1/1
20294/20294 [==============================] - 40s 2ms/step - loss: 1.9426 - acc: 0.0410


In [ ]:
model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

In [ ]:
for i in range(1000,1210):
    test = encoder_input[i:i+1,:,:]#i:i+1保持数组是三维
    out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    print(input_texts[i])
    print(out)
